***Техническое задание***

Задача состоит в создании модели машинного обучения для прогнозирования целевых переменных

Base: Эта выборка содержит данные о различных объектах. Первый признак в каждой строке представляет собой уникальный идентификатор объекта, а остальные 72 признака - числовые характеристики объектов.

Train: Этот набор данных предназначен для обучения модели машинного обучения.

Validation: Этот набор данных предназначен для оценки качества обученной модели.

Validation_answer: Этот набор данных содержит информацию о правильных ответах (целевых переменных) для Base

Цель состоит в том, чтобы обучить модель на данных из Train выборки, прогнозировать целевые переменные для Base выборки и оценить точность прогнозов, используя Validation_answer посредством метрики accuracy@5.

***Глобальные переменные***

In [1]:
RANDOM_STATE = 12345

***Предобработка данных***

Подгружаю необходимые библиотеки

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import faiss

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer

Прочитаю данные

In [3]:
base = pd.read_csv('base.csv', index_col=0)

In [4]:
train = pd.read_csv('train.csv', index_col=0)

In [5]:
validation = pd.read_csv('validation.csv', index_col=0)

In [6]:
validation_answer = pd.read_csv('validation_answer.csv', index_col=0)

Посмотрю данные

In [7]:
base.head(5)

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
Id,,,,,,,,,,,,,,,,,,,,,
0-base,-115.083890,11.152912,-64.426760,-118.88089,216.482440,-104.698060,-469.070588,44.348083,120.915344,181.449700,...,-42.808693,38.800827,-151.76218,-74.389090,63.66634,-4.703861,92.933610,115.269190,-112.756640,-60.830353
1-base,-34.562202,13.332763,-69.787610,-166.53348,57.680607,-86.098370,-85.076666,-35.637436,119.718636,195.234190,...,-117.767525,41.100000,-157.82940,-94.446806,68.20211,24.346846,179.937930,116.834000,-84.888941,-59.524610
2-base,-54.233746,6.379371,-29.210136,-133.41383,150.895830,-99.435326,52.554795,62.381706,128.951450,164.381470,...,-76.397800,46.011803,-207.14442,127.325570,65.56618,66.325680,81.073490,116.594154,-1074.464888,-32.527206
3-base,-87.520130,4.037884,-87.803030,-185.06763,76.369540,-58.985165,-383.182845,-33.611237,122.031910,136.233580,...,-70.647940,-6.358921,-147.20105,-37.692750,66.20289,-20.566910,137.206940,117.474100,-1074.464888,-72.915490
4-base,-72.743850,6.522049,43.671265,-140.60803,5.820023,-112.074080,-397.711282,45.182500,122.167180,112.119064,...,-57.199104,56.642403,-159.35184,85.944724,66.76632,-2.505783,65.315285,135.051590,-1074.464888,0.319401


In [8]:
train.head(5)

,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,Target
Id,,,,,,,,,,,,,,,,,,,,,
0-query,-53.882748,17.971436,-42.117104,-183.93668,187.517490,-87.144930,-347.360606,38.307602,109.085560,30.413513,...,70.107360,-155.80257,-101.965943,65.903790,34.457500,62.642094,134.763600,-415.750254,-25.958572,675816-base
1-query,-87.776370,6.806268,-32.054546,-177.26039,120.803330,-83.810590,-94.572749,-78.433090,124.915900,140.331070,...,4.669178,-151.69771,-1.638704,68.170876,25.096191,89.974976,130.589630,-1035.092211,-51.276833,366656-base
2-query,-49.979565,3.841486,-116.118590,-180.40198,190.128430,-50.837620,26.943937,-30.447489,125.771164,211.607820,...,78.039764,-169.14620,82.144186,66.008220,18.400496,212.409730,121.931470,-1074.464888,-22.547178,1447819-base
3-query,-47.810562,9.086598,-115.401695,-121.01136,94.652840,-109.255410,-775.150134,79.186520,124.003100,242.650650,...,44.515266,-145.41675,93.990981,64.131350,106.061920,83.178760,118.277725,-1074.464888,-19.902788,1472602-base
4-query,-79.632126,14.442886,-58.903397,-147.05254,57.127068,-16.239529,-321.317964,45.984676,125.941284,103.392670,...,45.028910,-196.09207,-117.626337,66.926220,42.456170,77.621765,92.479930,-1074.464888,-21.149351,717819-base


In [9]:
validation.head(5)

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
Id,,,,,,,,,,,,,,,,,,,,,
100000-query,-57.372734,3.597752,-13.213642,-125.92679,110.74594,-81.279594,-461.003172,139.815720,112.880980,75.215750,...,-75.51302,52.830902,-143.43945,59.051935,69.28224,61.927513,111.59253,115.140656,-1099.130485,-117.079360
100001-query,-53.758705,12.790300,-43.268543,-134.41762,114.44991,-90.520130,-759.626065,63.995087,127.117905,53.128998,...,-79.44183,29.185436,-168.60590,-82.872443,70.76560,-65.975950,97.07716,123.391640,-744.442332,-25.009320
100002-query,-64.175095,-3.980927,-7.679249,-170.16093,96.44616,-62.377740,-759.626065,87.477554,131.270110,168.920320,...,-134.79541,37.368730,-159.66231,-119.232725,67.71044,86.002060,137.63641,141.081630,-294.052271,-70.969604
100003-query,-99.286860,16.123936,9.837166,-148.06044,83.69708,-133.729720,58.576403,-19.046660,115.042404,75.206730,...,-77.23611,44.100494,-132.53012,-106.318982,70.88396,23.577892,133.18396,143.252940,-799.363667,-89.392670
100004-query,-79.532920,-0.364173,-16.027431,-170.88495,165.45392,-28.291668,33.931936,34.411217,128.903980,102.086914,...,-123.77025,45.635944,-134.25893,13.735359,70.61763,15.332115,154.56812,101.700640,-1171.892332,-125.307890


In [10]:
validation_answer.head(5)

,Expected
Id,
100000-query,2676668-base
100001-query,91606-base
100002-query,472256-base
100003-query,3168654-base
100004-query,75484-base


In [11]:
base.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2918139 entries, 0-base to 4744766-base
Data columns (total 72 columns):
 #   Column  Dtype  
---  ------  -----  
 0   0       float64
 1   1       float64
 2   2       float64
 3   3       float64
 4   4       float64
 5   5       float64
 6   6       float64
 7   7       float64
 8   8       float64
 9   9       float64
 10  10      float64
 11  11      float64
 12  12      float64
 13  13      float64
 14  14      float64
 15  15      float64
 16  16      float64
 17  17      float64
 18  18      float64
 19  19      float64
 20  20      float64
 21  21      float64
 22  22      float64
 23  23      float64
 24  24      float64
 25  25      float64
 26  26      float64
 27  27      float64
 28  28      float64
 29  29      float64
 30  30      float64
 31  31      float64
 32  32      float64
 33  33      float64
 34  34      float64
 35  35      float64
 36  36      float64
 37  37      float64
 38  38      float64
 39  39      float64


In [12]:
base.isna().sum().sum()

0

In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 0-query to 99999-query
Data columns (total 73 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       100000 non-null  float64
 1   1       100000 non-null  float64
 2   2       100000 non-null  float64
 3   3       100000 non-null  float64
 4   4       100000 non-null  float64
 5   5       100000 non-null  float64
 6   6       100000 non-null  float64
 7   7       100000 non-null  float64
 8   8       100000 non-null  float64
 9   9       100000 non-null  float64
 10  10      100000 non-null  float64
 11  11      100000 non-null  float64
 12  12      100000 non-null  float64
 13  13      100000 non-null  float64
 14  14      100000 non-null  float64
 15  15      100000 non-null  float64
 16  16      100000 non-null  float64
 17  17      100000 non-null  float64
 18  18      100000 non-null  float64
 19  19      100000 non-null  float64
 20  20      100000 non-null  float64
 21  21  

In [14]:
train.isna().sum().sum()

0

In [15]:
validation.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 100000-query to 199999-query
Data columns (total 72 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       100000 non-null  float64
 1   1       100000 non-null  float64
 2   2       100000 non-null  float64
 3   3       100000 non-null  float64
 4   4       100000 non-null  float64
 5   5       100000 non-null  float64
 6   6       100000 non-null  float64
 7   7       100000 non-null  float64
 8   8       100000 non-null  float64
 9   9       100000 non-null  float64
 10  10      100000 non-null  float64
 11  11      100000 non-null  float64
 12  12      100000 non-null  float64
 13  13      100000 non-null  float64
 14  14      100000 non-null  float64
 15  15      100000 non-null  float64
 16  16      100000 non-null  float64
 17  17      100000 non-null  float64
 18  18      100000 non-null  float64
 19  19      100000 non-null  float64
 20  20      100000 non-null  float64
 21

In [16]:
validation.isna().sum().sum()

0

In [17]:
validation_answer.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 100000-query to 199999-query
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Expected  100000 non-null  object
dtypes: object(1)
memory usage: 1.5+ MB


In [18]:
validation_answer.isna().sum().sum()

0

Пропусков нет

In [19]:
target = train['Target']

In [20]:
df_train = train.drop("Target", axis=1).copy()

Разделю train выборку на таргет и без таргет

In [21]:
index_col_base = base.index
index_col_train = train.index
index_col_validation = validation.index

Копирую индексы строк выборок

***Выбор модели FAISS***

In [22]:
numeric_features = base.select_dtypes(include=['float']).columns

In [23]:
standard_scaler = StandardScaler()
standard_scaler.fit(base)

StandardScaler()

In [24]:
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(base)

MinMaxScaler()

In [25]:
robust_scaler = RobustScaler()
robust_scaler.fit(base)

RobustScaler()

In [26]:
quantile_scaler = QuantileTransformer()
quantile_scaler.fit(base)

QuantileTransformer()

Я предлагаю попробовать различные варианты скейлеров и определить, при каком скейлере будет максимальная метрика точности acuracy@5

*Спойлер* робуст и квантиль скейлеры у меня запустить не получилось

In [27]:
scalers_list = [standard_scaler, min_max_scaler, robust_scaler, quantile_scaler]

Создаю лист из скейлеров

In [28]:
scaled_base_list = []
scaled_train_list = []
scaled_validation_list = []

Подготавливаю списки для масштабированных, посредством разных скейлеров, дат

In [29]:
def scale_data(data, scaler):
    scaled_array = scaler.transform(data)
    scaled_df = pd.DataFrame(scaled_array, columns=data.columns)
    scaled_df.index = data.index
    return scaled_df

Функиция масштабирования

In [30]:
%%time
for scaler in scalers_list:
    scaled_base_list.append(scale_data(base, scaler))
    scaled_train_list.append(scale_data(df_train, scaler))
    scaled_validation_list.append(scale_data(validation, scaler))

CPU times: total: 38.5 s
Wall time: 38.6 s


Заполнение списков масштабированными датами

In [31]:
dimensionality = base.shape[1]
dimensionality
quantizer = faiss.IndexFlatL2(dimensionality)

Определение кол-ва векторов для FAISS

In [32]:
def i_love_faiss_so_much(scaled_list, validation_answer):
    best_acc_5 = 0
    best_params = []

    for scaled_data in scaled_list:
        for i, scaled_base in enumerate(scaled_base_list):
            for nlist in range(41, 201, 10):
                acc = 0
                index_ivf = faiss.IndexIVFFlat(quantizer, dimensionality, nlist)
                index_ivf.train(np.ascontiguousarray(scaled_base.sample(frac=0.05, random_state=RANDOM_STATE).values).astype('float32'))
                index_ivf.add(np.ascontiguousarray(scaled_base.values).astype('float32'))
                base_index = {n: v for n, v in enumerate(scaled_base.index.to_list())}
                r, idx = index_ivf.search(scaled_data, 5)
                        
                for target, el in zip(validation_answer.values.tolist(), idx.tolist()):
                    acc += int(target in [base_index[r] for r in el])
                acc_5 = 100 * acc / len(idx)
            
                if best_acc_5 < acc_5:
                    best_acc_5 = acc_5
                    best_params = [acc_5, i, nlist]
                
                print('Accuracy@5 = ', acc_5, ' ,scaled_data = ', i, ' ,nlist = ', nlist)

    return best_acc_5, best_params

Функция подбора лучших гипперпараметров FAISS для определения максимальной метрики accuracy@5

Сейчас, при одготовке отчета, я вижу недочет в  написанной функции, безусловно нет необходимости проверять 

In [33]:
%%time
best_acc_5, best_params = i_love_faiss_so_much(scaled_train_list, target)

Accuracy@5 =  54.728  ,scaled_data =  0  ,nlist =  41
Accuracy@5 =  55.653  ,scaled_data =  0  ,nlist =  51
Accuracy@5 =  54.231  ,scaled_data =  0  ,nlist =  61
Accuracy@5 =  54.805  ,scaled_data =  0  ,nlist =  71
Accuracy@5 =  55.313  ,scaled_data =  0  ,nlist =  81
Accuracy@5 =  55.129  ,scaled_data =  0  ,nlist =  91
Accuracy@5 =  54.946  ,scaled_data =  0  ,nlist =  101
Accuracy@5 =  54.119  ,scaled_data =  0  ,nlist =  111
Accuracy@5 =  54.932  ,scaled_data =  0  ,nlist =  121
Accuracy@5 =  54.924  ,scaled_data =  0  ,nlist =  131
Accuracy@5 =  54.731  ,scaled_data =  0  ,nlist =  141
Accuracy@5 =  54.842  ,scaled_data =  0  ,nlist =  151
Accuracy@5 =  54.798  ,scaled_data =  0  ,nlist =  161
Accuracy@5 =  55.366  ,scaled_data =  0  ,nlist =  171
Accuracy@5 =  54.59  ,scaled_data =  0  ,nlist =  181
Accuracy@5 =  54.825  ,scaled_data =  0  ,nlist =  191
Accuracy@5 =  10.202  ,scaled_data =  1  ,nlist =  41
Accuracy@5 =  7.975  ,scaled_data =  1  ,nlist =  51
Accuracy@5 =  9.168 

In [40]:
print('best params = ', best_params)

best params =  [57.728, 3, 51]


Лучшая модель: скейлер - robust_scaler, количество класстеров - 51, лучшая acc@5 - 57,73%

<div class="alert alert-block alert-success">

Молодец, что подобрал наилучшие параметры  👍👍👍
    
</div>

In [43]:
%%time
nlist = 51
scaled_base = scaled_base_list[3]
scaled_data = scaled_train_list[3]

acc = 0
index_ivf = faiss.IndexIVFFlat(quantizer, dimensionality, nlist)
index_ivf.train(np.ascontiguousarray(scaled_base.sample(frac=0.05, random_state=RANDOM_STATE).values).astype('float32'))
index_ivf.add(np.ascontiguousarray(scaled_base.values).astype('float32'))
base_index = {n: v for n, v in enumerate(scaled_base.index.to_list())}
r, idx = index_ivf.search(scaled_data, 5)
                        
for true_target, el in zip(target.values.tolist(), idx.tolist()):
    acc += int(true_target in [base_index[r] for r in el])
acc_5 = 100 * acc / len(idx)

print('Accuracy@5 = ', acc_5,' ,nlist = ', nlist)

Accuracy@5 =  57.728  ,nlist =  51
CPU times: total: 6min 54s
Wall time: 1min 57s


Обучение на тренировочной выборке при лучших гипперпараметрах

In [44]:
validation_target = validation_answer['Expected']

Определения таргета в виде Series

In [45]:
%%time
scaled_valid_data = scaled_validation_list[3]

acc = 0
r, idx = index_ivf.search(scaled_valid_data, 5)
                        
for true_target, el in zip(validation_target.values.tolist(), idx.tolist()):
    acc += int(true_target in [base_index[r] for r in el])
acc_5 = 100 * acc / len(idx)

print('Accuracy@5 = ', acc_5,)

Accuracy@5 =  57.422
CPU times: total: 6min 40s
Wall time: 1min 57s


Определение таргетов и точности отработки validation дата на base

***Выводы***

В ходе выполнения работы было:
1) Открыты и проверены данные на наличие пропусков
2) Разработана модель на библиотеке FAISS.
3) Лучшая комбинация гипперпараметров модели позволяет определить правильных соседей для элементов валидационной выборки с точносью accuracy@5 ~57%

Это не лучший показатель, я уверен, что его можно улучшить. Вот несколько вариантов:
1) Рассмотреть другие комбинации гипперпараметров
2) Рассмотреть другие скейлеры
3) Предобработать данные на выбросы
4) Использовать другой метод расчета расстояний между векторами